# Azure ML Model Monitoring Demo - Model Deployment

Series of sample notebooks designed to showcase [AML's continuous model monitoring capabilities](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-monitor-model-performance?view=azureml-api-2&tabs=azure-cli). The series of notebooks in this repo have been developed to perform core operations including model training, deployment, simulated production data scoring, and inference data collection. These notebooks have been designed to be run in order and include the following steps:

- 00. Data Upload - Load time-series weather data from a local CSV into an AML datastore, and register as training & evaluation datasets
- 01. Model Training - Train a custom temperature prediction regression model using Mlflow & Scikit-Learn and register into your AML workspace
- <b>02. Model Deployment - Deploy your newly trained model to a Managed Online Endpoint with production data collection configured.</b>
- 03. Production Data Simulation - Send time-series data to your endpoint at a slow rate to simulate production inferencing. All submitted data will be collected automatically.
- 04. Monitoring Configuration - Configure a production model data monitor looking for drift in inferencing data, and scored results which can indicate that retraining should be performed.
- 05. Offline Monitoring - Sample notebook showcasing how to identify drift in data from datasets scored outside of Azure ML.

<b>This notebook utilizes the previously registered `Temperature_Prediction_Model` and deploys it to a Managed Online Endpoint. These endpoints are supported by the Azure Machine Learning service and offer dynamic, rule-based, scaling on managed Azure infrastructure. Additionally, they support automatic [model data collection](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-collect-production-data?view=azureml-api-2&tabs=azure-cli) which can be used to collect production data that can be subjected to drift analysis and used as a triggering mechanism to kick-off retraining operations.</b>

### Import required packages

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Environment, CodeConfiguration, DataCollector, DeploymentCollection
from azure.identity import DefaultAzureCredential
from mlflow import set_tracking_uri
import mlflow

### Establish connection to AML workspace using the v2 SDK

In [ ]:
subscription_id = "<your_subscription_id>"
resource_group = "<your_resource_group>"
workspace_name = "<your_workspace_name>"

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace_name)
workspace = ml_client.workspaces.get(workspace_name)
tracking_uri = workspace.mlflow_tracking_uri

set_tracking_uri(tracking_uri)

### Create managed online endpoint

In [ ]:
endpoint_name = "temp-pred-endpoint"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Weather prediction model",
    auth_mode="key",
)

ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### Retrieve registered model from AML workspace

In [ ]:
model = None
for m in ml_client.models.list('Temperature_Prediction_Model'):
    model = m
    break
    
model

### Define inferencing endpoint environment (Conda YAML file contains additional dependencies for model data collection utilities)

In [ ]:
environment = Environment(
    conda_file="deployment/model/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    name="temperature-prediction-env"
)

ml_client.environments.create_or_update(environment)

### Create deployment with `DataCollector` settings configured to capture inputs, outputs, and inputs/outputs jointly on a minute-by-minute basis.

The captured data will be collected as JSONL files within your AML datastore. Additionally, the custom `score.py` file has been designed to capture desired inputs & outputs.

In [ ]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="deployment/src", scoring_script="score.py"
    ),
    instance_type="Standard_F4s_v2",
    instance_count=1,
    data_collector = DataCollector(rolling_rate='minute', collections={'model_inputs': DeploymentCollection(enabled='true'), 
                                                'model_outputs': DeploymentCollection(enabled='true'), 
                                                'model_inputs_outputs': DeploymentCollection(enabled='true')})
)

### Push deployment to endpoint and await completion

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

### Test your model

Use the sample data below inside the AML studio to validate model is returning results as expected.

```
{
    "data":[{"latitude": 30.349, "longitude": -85.788, "elevation": 21.0, "windAngle": 140.0, "windSpeed": 5.1, "seaLvlPressure": 1016.3, "presentWeatherIndicator": 3.0, "pastWeatherIndicator": 2.0, "precipTime": 1.0, "precipDepth": 0.0, "snowDepth": 3.0, "year": 2019.0, "day": 1.0, "version": 1.0, "hour": 0.0, "month": 1.0, "weekday": 1.0, "usaf": 720735, "wban": 73805, "cloudCoverage": "NONE", "stationName": "NORTHWEST FLORIDA BEACHES INTL ARPT", "countryOrRegion": "US", "p_k": "720735-73805"}, {"latitude": 30.349, "longitude": -85.788, "elevation": 21.0, "windAngle": 150.0, "windSpeed": 5.7, "seaLvlPressure": 1016.3, "presentWeatherIndicator": 3.0, "pastWeatherIndicator": 2.0, "precipTime": 1.0, "precipDepth": 0.0, "snowDepth": 3.0, "year": 2019.0, "day": 1.0, "version": 1.0, "hour": 0.0, "month": 1.0, "weekday": 1.0, "usaf": 720735, "wban": 73805, "cloudCoverage": "NONE", "stationName": "NORTHWEST FLORIDA BEACHES INTL ARPT", "countryOrRegion": "US", "p_k": "720735-73805"}, {"latitude": 30.349, "longitude": -85.788, "elevation": 21.0, "windAngle": 150.0, "windSpeed": 4.6, "seaLvlPressure": 1019.5, "presentWeatherIndicator": 3.0, "pastWeatherIndicator": 2.0, "precipTime": 1.0, "precipDepth": 0.0, "snowDepth": 3.0, "year": 2019.0, "day": 1.0, "version": 1.0, "hour": 0.0, "month": 1.0, "weekday": 1.0, "usaf": 720735, "wban": 73805, "cloudCoverage": "NONE", "stationName": "NORTHWEST FLORIDA BEACHES INTL ARPT", "countryOrRegion": "US", "p_k": "720735-73805"}]
}
```